In [1]:
# Instalar dependencias necesarias (sin xvfb ni pyvirtualdisplay)
!pip install pyglet==1.5.1
!pip install gym==0.24
!pip install pygame
!pip install numpy
!pip install imageio imageio_ffmpeg

In [2]:
!pip install gym==0.24
!pip install pygame
!pip install numpy
!pip install tqdm
!pip install imageio imageio_ffmpeg

In [3]:
import numpy as np
import gym
import random
import imageio
from tqdm.notebook import trange
from tqdm import trange
import pandas as pd
import os
import imageio
from IPython.display import Image

In [10]:
import numpy as np
import pandas as pd

# Cargar el dataset desde el archivo CSV
dataset = pd.read_csv('futbol_real_dataset.csv')

# Mostrar el dataset para verificar que se cargó correctamente
print("Dataset de fútbol cargado desde CSV:\n", dataset)

# Definir los estados y acciones únicos
states = dataset['state'].unique().tolist()
actions = dataset['action'].unique()

# Añadir el estado 'Reinicio' si no está en el dataset
if 'Reinicio' not in states:
    states.append('Reinicio')

# Inicializamos la tabla Q con ceros
Qtable = np.zeros((len(states), len(actions)))

# Mapear estados y acciones a índices
state_to_index = {state: idx for idx, state in enumerate(states)}
action_to_index = {action: idx for idx, action in enumerate(actions)}

# Parámetros del Q-learning
alpha = 0.7   # Tasa de aprendizaje
gamma = 0.95  # Factor de descuento

# Entrenamiento de Q-learning usando el dataset cargado
for i, row in dataset.iterrows():
    state_idx = state_to_index[row['state']]
    action_idx = action_to_index[row['action']]
    next_state_idx = state_to_index[row['next_state']]

    # Obtener la recompensa
    reward = row['reward']

    # Actualizar la tabla Q usando la fórmula de Q-learning
    Qtable[state_idx, action_idx] = Qtable[state_idx, action_idx] + alpha * (
        reward + gamma * np.max(Qtable[next_state_idx, :]) - Qtable[state_idx, action_idx]
    )

# Mostrar la tabla Q entrenada
print("\nTabla Q entrenada:")
print(Qtable)

# Mostrar el mapeo de índices para entender mejor los resultados
print("\nEstado -> Índice:", state_to_index)
print("Acción -> Índice:", action_to_index)


Dataset de fútbol cargado desde CSV:
            state   action  reward    next_state
0    Ataque_zona  Disparo       1      Reinicio
1    Ataque_zona     Pase       0   Ataque_zona
2    Medio_campo     Pase       0   Ataque_zona
3    Medio_campo  Despeje       0  Defensa_zona
4   Defensa_zona  Despeje      -1   Medio_campo
5   Defensa_zona     Pase       0   Ataque_zona
6    Ataque_zona  Disparo       1      Reinicio
7    Ataque_zona     Pase       0   Medio_campo
8    Medio_campo     Pase       0   Ataque_zona
9    Medio_campo  Despeje      -1  Defensa_zona
10  Defensa_zona  Despeje      -1   Medio_campo
11  Defensa_zona     Pase       0   Ataque_zona
12   Ataque_zona     Pase       0   Medio_campo
13   Medio_campo  Despeje       0  Defensa_zona
14  Defensa_zona  Despeje      -1   Medio_campo

Tabla Q entrenada:
[[ 0.91        0.63005425  0.        ]
 [ 0.          0.7448      0.37815925]
 [ 0.          0.7448     -0.30126023]
 [ 0.          0.          0.        ]]

Estado -> Índice

In [ ]:
--------------------------------------------------------------------------------------------------------------------------

In [6]:
import pandas as pd

# Creamos un dataset estructurado para un partido de fútbol con más datos
data = {
    'state': ['Ataque_zona', 'Medio_campo', 'Defensa_zona', 'Ataque_zona', 'Medio_campo',
              'Defensa_zona', 'Ataque_zona', 'Ataque_zona', 'Medio_campo', 'Defensa_zona',
              'Medio_campo', 'Ataque_zona', 'Medio_campo', 'Defensa_zona', 'Medio_campo'],
    'action': ['Disparo', 'Pase', 'Despeje', 'Pase', 'Pase',
               'Despeje', 'Disparo', 'Pase', 'Pase', 'Despeje',
               'Pase', 'Disparo', 'Pase', 'Despeje', 'Pase'],
    'reward': [1, 0, 0, 0, 0, 
               0, 1, 0, 0, 0, 
               0, 1, 0, 0, 0],  # Recompensa +1 por un gol
    'next_state': ['Reinicio', 'Ataque_zona', 'Medio_campo', 'Ataque_zona', 'Ataque_zona',
                   'Medio_campo', 'Reinicio', 'Ataque_zona', 'Ataque_zona', 'Medio_campo',
                   'Ataque_zona', 'Reinicio', 'Defensa_zona', 'Medio_campo', 'Ataque_zona']
}

# Convertimos el diccionario en un DataFrame de pandas
df = pd.DataFrame(data)

# Guardamos el DataFrame como un archivo CSV
df.to_csv('futbol_dataset.csv', index=False)

print("Archivo CSV generado: 'futbol_dataset.csv'")


Archivo CSV generado: 'futbol_dataset.csv'


In [9]:
import pandas as pd

# Simulamos un dataset estructurado para un partido de fútbol
data = {
    'state': ['Ataque_zona', 'Ataque_zona', 'Medio_campo', 'Medio_campo', 'Defensa_zona', 'Defensa_zona', 'Ataque_zona', 'Ataque_zona',
              'Medio_campo', 'Medio_campo', 'Defensa_zona', 'Defensa_zona', 'Ataque_zona', 'Medio_campo', 'Defensa_zona'],
    'action': ['Disparo', 'Pase', 'Pase', 'Despeje', 'Despeje', 'Pase', 'Disparo', 'Pase', 'Pase', 'Despeje', 'Despeje', 'Pase', 'Pase', 'Despeje', 'Despeje'],
    'reward': [1, 0, 0, 0, -1, 0, 1, 0, 0, -1, -1, 0, 0, 0, -1],  # Recompensa de +1 por gol, -1 por perder posesión, 0 por acciones sin impacto
    'next_state': ['Reinicio', 'Ataque_zona', 'Ataque_zona', 'Defensa_zona', 'Medio_campo', 'Ataque_zona', 'Reinicio', 'Medio_campo',
                   'Ataque_zona', 'Defensa_zona', 'Medio_campo', 'Ataque_zona', 'Medio_campo', 'Defensa_zona', 'Medio_campo']
}

# Convertimos el diccionario en un DataFrame de pandas
df = pd.DataFrame(data)

# Guardamos el DataFrame como un archivo CSV para que lo uses más tarde
df.to_csv('futbol_real_dataset.csv', index=False)

# Mostrar el dataset
print("Dataset creado:\n", df)


Dataset creado:
            state   action  reward    next_state
0    Ataque_zona  Disparo       1      Reinicio
1    Ataque_zona     Pase       0   Ataque_zona
2    Medio_campo     Pase       0   Ataque_zona
3    Medio_campo  Despeje       0  Defensa_zona
4   Defensa_zona  Despeje      -1   Medio_campo
5   Defensa_zona     Pase       0   Ataque_zona
6    Ataque_zona  Disparo       1      Reinicio
7    Ataque_zona     Pase       0   Medio_campo
8    Medio_campo     Pase       0   Ataque_zona
9    Medio_campo  Despeje      -1  Defensa_zona
10  Defensa_zona  Despeje      -1   Medio_campo
11  Defensa_zona     Pase       0   Ataque_zona
12   Ataque_zona     Pase       0   Medio_campo
13   Medio_campo  Despeje       0  Defensa_zona
14  Defensa_zona  Despeje      -1   Medio_campo


**Desde aqui se hace**

**Desde aqui se hace**

In [19]:
import pandas as pd
import numpy as np

# Cargar el dataset
file_path = 'england-premier-league-matches-2018-to-2019-stats.csv'
matches_df = pd.read_csv(file_path)

# Mostrar las primeras filas del dataset para entender la estructura
print(matches_df.head())

# Definir los estados usando columnas clave
matches_df['state'] = matches_df.apply(lambda row: (
    row['home_team_goal_count'], 
    row['away_team_goal_count'], 
    row['home_team_possession'], 
    row['away_team_possession'], 
    row['home_team_shots_on_target'], 
    row['away_team_shots_on_target']
), axis=1)

# Definir las acciones: Atacar, Defender, Jugar equilibrado (para simplificar)
actions = ['Atacar', 'Defender', 'Equilibrado']

# Mapear estados y acciones a índices
state_list = matches_df['state'].unique().tolist()
state_to_index = {state: idx for idx, state in enumerate(state_list)}
action_to_index = {action: idx for idx, action in enumerate(actions)}

# Inicializamos la tabla Q
Qtable = np.zeros((len(state_list), len(actions)))

# Parámetros del Q-learning
alpha = 0.7   # Tasa de aprendizaje
gamma = 0.95  # Factor de descuento

# Recompensa basada en el resultado del partido
def calculate_reward(row):
    if row['home_team_goal_count'] > row['away_team_goal_count']:
        return 3  # Victoria
    elif row['home_team_goal_count'] == row['away_team_goal_count']:
        return 1  # Empate
    else:
        return -1  # Derrota

# Aplicamos la recompensa
matches_df['reward'] = matches_df.apply(calculate_reward, axis=1)

# Entrenamiento de Q-learning usando el dataset
for i, row in matches_df.iterrows():
    state_idx = state_to_index[row['state']]
    # Aquí se podría seleccionar una acción aleatoria o basada en una política epsilon-greedy
    action_idx = np.random.choice(len(actions))  # Para simplificar, elegimos una acción al azar

    # Obtener la recompensa
    reward = row['reward']

    # Actualizar la tabla Q usando la fórmula de Q-learning
    Qtable[state_idx, action_idx] = Qtable[state_idx, action_idx] + alpha * (
        reward + gamma * np.max(Qtable[state_idx, :]) - Qtable[state_idx, action_idx]
    )

# Mostrar la tabla Q entrenada
print("\nTabla Q entrenada:")
print(Qtable)

# Mostrar el mapeo de estados y acciones
print("\nEstado -> Índice:", state_to_index)
print("Acción -> Índice:", action_to_index)


    timestamp               date_GMT    status  attendance     home_team_name  \
0  1533927600   Aug 10 2018 - 7:00pm  complete       74439  Manchester United   
1  1533987000  Aug 11 2018 - 11:30am  complete       51749   Newcastle United   
2  1533996000   Aug 11 2018 - 2:00pm  complete       10353    AFC Bournemouth   
3  1533996000   Aug 11 2018 - 2:00pm  complete       24821             Fulham   
4  1533996000   Aug 11 2018 - 2:00pm  complete       24121  Huddersfield Town   

      away_team_name          referee  Game Week  Pre-Match PPG (Home)  \
0     Leicester City   Andre Marriner          1                   0.0   
1  Tottenham Hotspur  Martin Atkinson          1                   0.0   
2       Cardiff City     Kevin Friend          1                   0.0   
3     Crystal Palace        Mike Dean          1                   0.0   
4            Chelsea   Chris Kavanagh          1                   0.0   

   Pre-Match PPG (Away)  ...  odds_ft_home_team_win  odds_ft_draw  \

In [20]:
import pandas as pd

def print_qtable_explanatory(Qtable, state_to_index, action_to_index):
    # Crear un DataFrame con los valores de la tabla Q
    actions = list(action_to_index.keys())  # Lista de acciones
    df_qtable = pd.DataFrame(Qtable, columns=actions)

    # Crear una lista para mostrar los estados de manera más legible
    state_list = list(state_to_index.keys())
    df_qtable.insert(0, 'State', state_list)  # Insertar los estados como primera columna

    # Mostrar la tabla Q de forma más explicativa
    return df_qtable

# Crear la tabla explicativa
explanatory_qtable = print_qtable_explanatory(Qtable, state_to_index, action_to_index)

# Mostrar la tabla
print(explanatory_qtable)
# Ejemplo: (goles local, goles visitante, posesión local, posesión visitante, disparos local, disparos visitante)

                     State  Atacar  Defender  Equilibrado
0     (2, 1, 46, 54, 7, 5)     0.0       2.1          0.0
1     (1, 2, 40, 60, 3, 6)     0.0      -0.7          0.0
2     (2, 0, 62, 38, 5, 2)     0.0       2.1          0.0
3    (0, 2, 66, 34, 7, 11)    -0.7       0.0          0.0
4     (0, 3, 37, 63, 2, 5)     0.0      -0.7          0.0
..                     ...     ...       ...          ...
375   (2, 0, 59, 41, 6, 3)     2.1       0.0          0.0
376  (0, 2, 73, 27, 11, 5)     0.0       0.0         -0.7
377   (1, 1, 53, 47, 4, 4)     0.0       0.7          0.0
378  (2, 2, 55, 45, 4, 10)     0.0       0.7          0.0
379  (1, 4, 48, 52, 9, 10)     0.0       0.0         -0.7

[380 rows x 4 columns]


In [25]:
def predict_action(Qtable, state_to_index, action_to_index, nuevo_estado):
    # Verificar si el estado está en la tabla de estados
    if nuevo_estado in state_to_index:
        state_idx = state_to_index[nuevo_estado]
        # Encontrar la acción con la recompensa más alta
        best_action_idx = np.argmax(Qtable[state_idx])
        best_action = list(action_to_index.keys())[best_action_idx]
        print(f"Para el estado {nuevo_estado}, la mejor acción es: {best_action}")
    else:
        # Si el estado no está en la tabla Q, elegir una acción aleatoria o por defecto
        best_action = np.random.choice(list(action_to_index.keys()))  # O puedes elegir 'Equilibrado' como acción predeterminada
        print(f"Estado {nuevo_estado} no encontrado en la tabla Q. Se elige la acción '{best_action}' por defecto.")

# Ejemplo de uso para predicción con estado desconocido
nuevo_estado = (6, 2, 78, 55, 6, 6)  # Un estado nuevo que no fue parte del entrenamiento
predict_action(Qtable, state_to_index, action_to_index, nuevo_estado)



Estado (6, 2, 78, 55, 6, 6) no encontrado en la tabla Q. Se elige la acción 'Atacar' por defecto.
